# CDC 500 Cities and the COVID-19 Outbreak
### Import CDC 500 cities data

In [1]:
import pandas as pd; import numpy as np
df = pd.read_csv('500_Cities__Local_Data_for_Better_Health__2019_release.csv')#.dropna(subset=['TractFIPS'])
df

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2016,IL,Illinois,Chicago,Census Tract,BRFSS,Health Outcomes,1714000-17031612000,All teeth lost among adults aged >=65 Years,%,...,39.9,NaN,NaN,"1,545","(41.7978467034, -87.6525358919)",HLTHOUT,TEETHLOST,1714000.0,1.703161e+10,Teeth Loss
1,2016,IL,Illinois,Chicago,Census Tract,BRFSS,Prevention,1714000-17031660700,Visits to dentist or dental clinic among adult...,%,...,46.3,NaN,NaN,"2,366","(41.7738453587, -87.6810954614)",PREVENT,DENTAL,1714000.0,1.703166e+10,Dental Visit
2,2017,IL,Illinois,Decatur,Census Tract,BRFSS,Health Outcomes,1718823-17115001100,Current asthma among adults aged >=18 Years,%,...,10.5,NaN,NaN,"3,485","(39.8496654386, -88.9073271617)",HLTHOUT,CASTHMA,1718823.0,1.711500e+10,Current Asthma
3,2016,IL,Illinois,Joliet,Census Tract,BRFSS,Health Outcomes,1738570-17197881603,All teeth lost among adults aged >=65 Years,%,...,27.4,NaN,NaN,"3,301","(41.5405415324, -88.133133385)",HLTHOUT,TEETHLOST,1738570.0,1.719788e+10,Teeth Loss
4,2017,IL,Illinois,Elgin,Census Tract,BRFSS,Health Outcomes,1723074-17031804405,High cholesterol among adults aged >=18 Years ...,%,...,26.3,NaN,NaN,"3,632","(42.0349418585, -88.2462502287)",HLTHOUT,HIGHCHOL,1723074.0,1.703180e+10,High Cholesterol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810098,2017,TX,Texas,El Paso,Census Tract,BRFSS,Health Outcomes,4824000-48141000600,Diagnosed diabetes among adults aged >=18 Years,%,...,19.5,NaN,NaN,"4,727","(31.8322684956, -106.448506723)",HLTHOUT,DIABETES,4824000.0,4.814100e+10,Diabetes
810099,2017,TX,Texas,El Paso,Census Tract,BRFSS,Health Outcomes,4824000-48141003000,Mental health not good for >=14 days among adu...,%,...,18.7,NaN,NaN,"4,188","(31.7653840882, -106.436891712)",HLTHOUT,MHLTH,4824000.0,4.814100e+10,Mental Health
810100,2017,TX,Texas,Fort Worth,Census Tract,BRFSS,Health Outcomes,4827000-48439111018,Stroke among adults aged >=18 Years,%,...,NaN,*,Estimates suppressed for population less than 50,6,"(32.5870865478, -97.4184209671)",HLTHOUT,STROKE,4827000.0,4.843911e+10,Stroke
810101,2016,TX,Texas,Fort Worth,Census Tract,BRFSS,Health Outcomes,4827000-48439113928,All teeth lost among adults aged >=65 Years,%,...,9.3,NaN,NaN,"8,308","(32.9680935802, -97.2611225917)",HLTHOUT,TEETHLOST,4827000.0,4.843911e+10,Teeth Loss


### Extracting City/County IDs from the UniqueID
#### Each UniqueID identifies a census tract by an 11 digit FIPS code.
#### Here we parse the UniqueID for the first 5 digits of this FIPS code, which will be used to identify the County of which that tract is a part. 

In [3]:
def get_coufips(fips):
    if fips:
        if len(fips.split('-')) == 1:
            return fips[:5]
        else:
            return fips.split('-')[1][:5] 
    else:
        return np.nan
df['COUFIPS'] = [get_coufips(fips) for fips in df.UniqueID]
df.dropna(subset=['COUFIPS'],inplace=True)

### Now we use the unique five digit FIPS codes to aggregate (median) the CDC data for each measure (2017) from Census tract level to county/city-level

In [7]:
coudf = pd.DataFrame({'COUFIPS':df.COUFIPS.unique()})

x = 0
for mid in df.MeasureId.unique():
    ndf = df[df.Year==2017][df.MeasureId==mid]
    print('Progress: {:.2%}'.format(x/len(df.MeasureId.unique())), end = '\r')
    if len(ndf) > 0:
        data_values = []
        for i in coudf.COUFIPS:
            if len(ndf[ndf.COUFIPS==i]) > 0:
                nndf = ndf[ndf.COUFIPS==i].reset_index()
                value = nndf.Data_Value.median()
                data_values.append(value)
            else:
                data_values.append(np.nan)
        coudf[mid] = data_values
    x+=1

metro_area = []
for i in coudf.COUFIPS:
    metro_area.append(str([x for x in df[df.COUFIPS==i].CityName.unique()]).replace("[","").replace("]",""))
coudf['METRO'] = metro_area
coudf

,COUFIPS,CASTHMA,HIGHCHOL,DIABETES,OBESITY,CANCER,STROKE,MHLTH,CSMOKING,CHOLSCREEN,...,CHECKUP,KIDNEY,BINGE,LPA,ARTHRITIS,BPMED,PHLTH,BPHIGH,COPD,METRO
0,17031,8.40,29.50,10.50,31.40,4.90,2.80,12.70,17.60,81.90,...,66.60,3.00,21.30,28.20,20.20,71.20,12.00,28.35,6.10,"'Chicago', 'Elgin', 'Palatine', 'Cicero', 'Eva..."
1,17115,10.40,36.30,11.30,37.40,7.20,3.70,14.50,21.00,82.10,...,69.70,3.40,17.20,27.80,29.50,76.50,14.00,37.00,8.70,'Decatur'
2,17197,9.35,31.15,9.65,34.30,4.80,2.55,12.60,17.85,82.65,...,70.55,2.60,20.45,24.30,22.15,67.35,11.00,31.40,5.80,"'Joliet', 'Naperville', 'Bolingbrook', 'Aurora'"
3,17143,10.65,33.20,10.70,37.00,5.90,3.40,14.60,20.25,80.30,...,68.75,3.25,18.55,28.10,26.65,73.35,12.95,34.90,7.60,"'Peoria', 'Cicero'"
4,17089,9.30,30.10,10.55,34.10,5.00,2.70,14.00,18.85,79.15,...,65.20,2.90,19.75,26.95,20.45,68.10,12.75,28.15,6.45,"'Elgin', 'Aurora'"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,48253,7.95,32.20,10.15,36.20,5.25,2.70,13.25,21.85,76.70,...,64.40,2.50,22.60,32.20,20.00,66.20,12.20,33.50,5.80,'Abilene'
810,48209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'Austin'
811,48355,7.90,34.85,13.55,34.05,5.40,3.10,13.20,16.00,78.65,...,67.60,3.30,18.65,35.15,23.30,74.30,12.70,34.50,5.70,'Corpus Christi'
812,48041,9.00,26.70,7.25,26.00,3.80,1.95,15.75,14.95,72.30,...,67.00,2.20,19.45,24.50,13.95,64.15,9.40,21.70,4.10,"'College Station', 'Bryan'"


## Import JHU time series data for US counties

In [10]:
url1 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url2 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url3 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
url4 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
url5 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
#conf = pd.read_csv(url1)
#dths = pd.read_csv(url2)
#recd = pd.read_csv(url3)
confUS = pd.read_csv(url4)
dthsUS = pd.read_csv(url5)

confUS = confUS[confUS.Long_!=0]
dthsUS = dthsUS[dthsUS.Long_!=0]
def get_fips(fips):
    fips = fips.replace('.0','')
    if len(fips) == 4:
        return '0'+fips
    else:
        return fips
confUS['FIPS'] = [get_fips(str(i)) for i in confUS.FIPS]
dthsUS['FIPS'] = [get_fips(str(i)) for i in dthsUS.FIPS]

In [11]:
for i in coudf.index:
    if coudf.COUFIPS[i] in [fips for fips in confUS['FIPS']]:
        print(str(i) + ' FIPS Code Match: ', end='')
        ndf = confUS[confUS.FIPS==coudf.COUFIPS[i]].reset_index()
        print(ndf['Admin2'][0] + ', ' + ndf['Province_State'][0])
        print('CDC 500 Metro Area: '+ coudf.METRO[i])
        print('------------------------------')

0 FIPS Code Match: Cook, Illinois
CDC 500 Metro Area: 'Chicago', 'Elgin', 'Palatine', 'Cicero', 'Evanston', 'Schaumburg', 'Arlington Heights'
------------------------------
1 FIPS Code Match: Macon, Illinois
CDC 500 Metro Area: 'Decatur'
------------------------------
2 FIPS Code Match: Will, Illinois
CDC 500 Metro Area: 'Joliet', 'Naperville', 'Bolingbrook', 'Aurora'
------------------------------
3 FIPS Code Match: Peoria, Illinois
CDC 500 Metro Area: 'Peoria', 'Cicero'
------------------------------
4 FIPS Code Match: Kane, Illinois
CDC 500 Metro Area: 'Elgin', 'Aurora'
------------------------------
6 FIPS Code Match: DuPage, Illinois
CDC 500 Metro Area: 'Naperville', 'Aurora', 'Bolingbrook'
------------------------------
8 FIPS Code Match: Winnebago, Illinois
CDC 500 Metro Area: 'Rockford'
------------------------------
16 FIPS Code Match: Kendall, Illinois
CDC 500 Metro Area: 'Joliet', 'Aurora'
------------------------------
17 FIPS Code Match: Jefferson, Alabama
CDC 500 Metro Ar

289 FIPS Code Match: Lancaster, Nebraska
CDC 500 Metro Area: 'Lincoln'
------------------------------
290 FIPS Code Match: Greene, Missouri
CDC 500 Metro Area: 'Springfield'
------------------------------
291 FIPS Code Match: Yellowstone, Montana
CDC 500 Metro Area: 'Billings'
------------------------------
292 FIPS Code Match: Christian, Missouri
CDC 500 Metro Area: 'Springfield'
------------------------------
299 FIPS Code Match: Musselshell, Montana
CDC 500 Metro Area: 'Billings'
------------------------------
306 FIPS Code Match: San Mateo, California
CDC 500 Metro Area: 'Redwood City', 'Daly City', 'San Mateo'
------------------------------
320 FIPS Code Match: Monterey, California
CDC 500 Metro Area: 'Salinas'
------------------------------
322 FIPS Code Match: Ventura, California
CDC 500 Metro Area: 'San Buenaventura (Ventura)', 'Simi Valley', 'Carlsbad', 'Oxnard', 'Thousand Oaks'
------------------------------
330 FIPS Code Match: Allegheny, Pennsylvania
CDC 500 Metro Area: 'Pi

592 FIPS Code Match: Tom Green, Texas
CDC 500 Metro Area: 'San Angelo'
------------------------------
593 FIPS Code Match: Medina, Texas
CDC 500 Metro Area: 'San Antonio'
------------------------------
594 FIPS Code Match: Williamson, Texas
CDC 500 Metro Area: 'Round Rock', 'Austin'
------------------------------
604 FIPS Code Match: Norfolk, Virginia
CDC 500 Metro Area: 'Norfolk'
------------------------------
605 FIPS Code Match: Utah, Utah
CDC 500 Metro Area: 'Orem', 'Provo'
------------------------------
606 FIPS Code Match: Alexandria, Virginia
CDC 500 Metro Area: 'Alexandria'
------------------------------
607 FIPS Code Match: Davis, Utah
CDC 500 Metro Area: 'Layton'
------------------------------
608 FIPS Code Match: Portsmouth, Virginia
CDC 500 Metro Area: 'Portsmouth'
------------------------------
609 FIPS Code Match: Hampton, Virginia
CDC 500 Metro Area: 'Hampton'
------------------------------
610 FIPS Code Match: Salt Lake, Utah
CDC 500 Metro Area: 'Salt Lake City', 'Sandy

In [16]:
cdc_covid_dths = pd.merge(coudf[['COUFIPS', 'METRO', 'CASTHMA', 'HIGHCHOL', 'DIABETES', 'OBESITY', 
                                 'CANCER', 'STROKE', 'MHLTH', 'CSMOKING', 'CHOLSCREEN', 
                                 'ACCESS2', 'CHD', 'CHECKUP', 'KIDNEY', 'BINGE', 'LPA', 
                                 'ARTHRITIS', 'BPMED', 'PHLTH', 'BPHIGH', 'COPD']], 
                          dthsUS.drop(columns=['UID', 'iso2', 'iso3', 'code3', 
                                               'Country_Region', 'Combined_Key']), 
                          left_on='COUFIPS',
                          right_on='FIPS', 
                          how='left')

cdc_covid_conf = pd.merge(coudf[['COUFIPS', 'METRO', 'CASTHMA', 'HIGHCHOL', 'DIABETES', 'OBESITY', 
                                 'CANCER', 'STROKE', 'MHLTH', 'CSMOKING', 'CHOLSCREEN', 
                                 'ACCESS2', 'CHD', 'CHECKUP', 'KIDNEY', 'BINGE', 'LPA', 
                                 'ARTHRITIS', 'BPMED', 'PHLTH', 'BPHIGH', 'COPD']], 
                          confUS.drop(columns=['UID', 'iso2', 'iso3', 'code3', 
                                               'Country_Region', 'Combined_Key']), 
                          left_on='COUFIPS',
                          right_on='FIPS', 
                          how='left')

In [26]:
cdc_covid_dths = cdc_covid_dths[['COUFIPS', 'METRO','FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', 'Population', 
                                 'CASTHMA', 'HIGHCHOL', 'DIABETES', 'OBESITY', 'CANCER', 'STROKE', 'MHLTH', 'CSMOKING', 
                                 'CHOLSCREEN', 'ACCESS2', 'CHD', 'CHECKUP', 'KIDNEY', 'BINGE', 'LPA', 'ARTHRITIS', 'BPMED', 
                                 'PHLTH', 'BPHIGH', 'COPD', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', 
                                 '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', 
                                 '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', 
                                 '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', 
                                 '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', 
                                 '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', 
                                 '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', 
                                 '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', 
                                 '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', 
                                 '4/5/20', '4/6/20']]
cdc_covid_dths.fillna('NAN', inplace=True)
cdc_covid_dths

,COUFIPS,METRO,FIPS,Admin2,Province_State,Lat,Long_,Population,CASTHMA,HIGHCHOL,...,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20
0,17031,"'Chicago', 'Elgin', 'Palatine', 'Cicero', 'Eva...",17031,Cook,Illinois,41.8414,-87.8166,5.15023e+06,8.4,29.5,...,28,40,44,61,95,107,141,167,186,209
1,17115,'Decatur',17115,Macon,Illinois,39.8606,-88.9634,104009,10.4,36.3,...,0,0,0,0,0,0,0,0,0,0
2,17197,"'Joliet', 'Naperville', 'Bolingbrook', 'Aurora'",17197,Will,Illinois,41.4462,-87.9786,690743,9.35,31.15,...,4,4,5,8,9,9,10,11,13,19
3,17143,"'Peoria', 'Cicero'",17143,Peoria,Illinois,40.787,-89.7608,179179,10.65,33.2,...,0,0,0,0,0,0,0,0,1,1
4,17089,"'Elgin', 'Aurora'",17089,Kane,Illinois,41.9384,-88.4286,532403,9.3,30.1,...,4,6,6,7,8,8,9,12,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,48253,'Abilene',48253,Jones,Texas,32.74,-99.8786,20083,7.95,32.2,...,0,0,0,0,0,0,0,0,0,0
810,48209,'Austin',48209,Hays,Texas,30.0567,-98.0294,230191,NAN,NAN,...,0,0,0,0,0,0,0,0,0,0
811,48355,'Corpus Christi',48355,Nueces,Texas,27.7363,-97.5433,362294,7.9,34.85,...,0,0,0,0,0,0,0,0,0,0
812,48041,"'College Station', 'Bryan'",48041,Brazos,Texas,30.6636,-96.3021,229211,9,26.7,...,1,2,2,2,3,4,5,5,6,6


In [27]:
cdc_covid_conf = cdc_covid_conf[['COUFIPS', 'METRO','FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', 
                                 'CASTHMA', 'HIGHCHOL', 'DIABETES', 'OBESITY', 'CANCER', 'STROKE', 'MHLTH', 'CSMOKING', 
                                 'CHOLSCREEN', 'ACCESS2', 'CHD', 'CHECKUP', 'KIDNEY', 'BINGE', 'LPA', 'ARTHRITIS', 'BPMED', 
                                 'PHLTH', 'BPHIGH', 'COPD', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', 
                                 '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', 
                                 '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', 
                                 '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', 
                                 '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', 
                                 '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', 
                                 '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', 
                                 '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', 
                                 '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', 
                                 '4/5/20', '4/6/20']]
cdc_covid_conf.fillna('-', inplace=True)
cdc_covid_conf

,COUFIPS,METRO,FIPS,Admin2,Province_State,Lat,Long_,CASTHMA,HIGHCHOL,DIABETES,...,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20
0,17031,"'Chicago', 'Elgin', 'Palatine', 'Cicero', 'Eva...",17031,Cook,Illinois,41.8414,-87.8166,8.4,29.5,10.5,...,2613,3445,3727,4496,5152,5575,6111,7439,8034,8728
1,17115,'Decatur',17115,Macon,Illinois,39.8606,-88.9634,10.4,36.3,11.3,...,1,2,2,2,2,2,2,5,6,9
2,17197,"'Joliet', 'Naperville', 'Bolingbrook', 'Aurora'",17197,Will,Illinois,41.4462,-87.9786,9.35,31.15,9.65,...,127,182,223,228,322,384,455,582,648,697
3,17143,"'Peoria', 'Cicero'",17143,Peoria,Illinois,40.787,-89.7608,10.65,33.2,10.7,...,7,8,8,8,10,11,11,11,14,12
4,17089,"'Elgin', 'Aurora'",17089,Kane,Illinois,41.9384,-88.4286,9.3,30.1,10.55,...,90,100,113,128,142,154,176,207,220,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,48253,'Abilene',48253,Jones,Texas,32.74,-99.8786,7.95,32.2,10.15,...,0,0,0,0,0,0,0,0,0,0
810,48209,'Austin',48209,Hays,Texas,30.0567,-98.0294,-,-,-,...,13,13,16,38,42,42,47,50,54,59
811,48355,'Corpus Christi',48355,Nueces,Texas,27.7363,-97.5433,7.9,34.85,13.55,...,21,22,22,34,34,40,41,47,53,59
812,48041,"'College Station', 'Bryan'",48041,Brazos,Texas,30.6636,-96.3021,9,26.7,7.25,...,40,44,46,53,63,68,72,75,94,94


In [28]:
from zipfile import ZipFile
filepaths = [(cdc_covid_conf,'covid-confirmed-500.csv'),(cdc_covid_dths,'covid-deaths-500.csv')]

with ZipFile('covid-500.zip','w') as zip: 
    # writing each file one by one 
    for data, filename  in filepaths: 
        data.to_csv(filename)
        zip.write(filename)